# K-EO601
The other script has some edits that this one could benefit from.


In [1]:
import pandas as pd
from functools import reduce
import time

import sys
import clr

sys.path.append(r'C:\Program Files (x86)\PIPC\AF\PublicAssemblies\4.0')
clr.AddReference('OSIsoft.AFSDK')
from OSIsoft.AF import *  
from OSIsoft.AF.PI import *  
from OSIsoft.AF.Asset import *  
from OSIsoft.AF.Data import *  
from OSIsoft.AF.Time import *  
from OSIsoft.AF.UnitsOfMeasure import *

In [5]:
pi_tags = ['PEO671A_EO2',
          'PEO671B_EO2',
          'TEO602_EO2',
          'PEO605_EO2',
          'PEO6411_EO2',
          'PEO6405_EO2',
          'PEO6404A_EO2',
          'PEO6404B_EO2',
          'PEO6406_EO2',
          # This is as far as I've gotten so far from the P&ID's
          # First 20 below are K-EO601 Seal System
          'PEO6407_EO2',
          'PEO6408_EO2',
          'XEO6401BZ_EO2',
          # 'PEO1810_EO2',
          'TEO6424_EO2',
          'PEO6402_EO2',
          'FEO6403_EO2',
          # 'PEO6403_EO2_OP',
          'PEO6403_EO2',
          # 'PEO6406_EO2',
          'PEO6409_EO2',
          'FEO6416_EO2',
          # 'FEO6416V_EO2',
          # 'PEO6405_EO2',
          # 'PEO6411_EO2',
          'FEO6417_EO2',
          'FEO6419_EO2',
          'PEO6415_EO2',
          'FEO6418_EO2',
          'PEO6413_EO2',
          'PEO6412_EO2',
          'PEO6414_EO2',
          'PEO6410_EO2', 
          'NEO6014X_EO2', # Vibration
          'NEO6014Y_EO2',
          'NEO6015X_EO2',
          'NEO6015Y_EO2',
          'TEO6402B_EO2', # Bearing Temps
          'TEO6403B_EO2', 
          'TEO6401B_EO2', 
          'TEO6407B_EO2', 
          'TEO6405A_EO2', 
          'TEO6413B_EO2', 
          'TEO6406B_EO2', 
          'TEO6408B_EO2', 
          'TEO6409B_EO2', 
          'TEO6410B_EO2', 
          'ZEO6028B_EO2',
          'NEO6016X_EO2',
          'NEO6013Y_EO2',
          'ZEO6030A_EO2',
          'ZEO6029A_EO2',
          'NEO6017X_EO2',
          'NEO6018Y_EO2',
          'FEO601_EO2',  # Methane Feed to recycle gas loop
          'TEO625_EO2', # loop suction temp
          'PEO602C_EO2', # Loop suction pressure ctl
          'LEO609_EO2', # Suction KO drum level
          'FEO504_EO2', # Discharge flowrate
          'PEO552_EO2', # Discharge pressure
          ]


# Add-on Pitags, 9/9/21
addon = ['TEO630_EO2',
         'EO2RECFLOW_EO2']

# Add-on Pitags 11/10/21
# Requested by C3
addon = ['XEO6401AZ_EO2',
         'FEO602_EO2',
         'PEO6414D_EO2',
         'SEO603_EO2',
         'SEO602_EO2',
         'TEO6422_EO2',
         'TEO6423_EO2',
         'JEO601_EO2',
         'NEO6012X_EO2',
         'NEO6012Y_EO2',
         'NEO6011X_EO2',
         'NEO6011Y_EO2'
        ]
         

pi_tags.extend(addon)

# Only fishing for the addon tags today..
pi_tags = addon

print(str(len(pi_tags)) + " pi-tags in the list")


12 pi-tags in the list


In [18]:
# time interval for the PI data
year = '2015'

# SHORT
start_time = year + '-01-1'
end_time = year+ '-12-31'

timerange = AFTimeRange(start_time, end_time)
span = AFTimeSpan.Parse("1m")

# Set the pi server
pi_server = 'DSAMPICOLL'
piServer = PIServers()[pi_server]

#pi_tags[:1]

In [19]:
start = time.perf_counter()

#df_list = []
#for tag in pi_tags[-6:]:
#for tag in pi_tags[:2]:
for tag in pi_tags:
    tic = time.perf_counter()
    point = PIPoint.FindPIPoint(piServer, tag)  
    name = point.Name.lower()
    #interpolated = point.InterpolatedValues(timerange, span, "", False)  
    recorded = point.RecordedValues(timerange, AFBoundaryType.Inside, "", False)
    
    df = pd.DataFrame(recorded.GetValueArrays(None, None, None)[1:3]).transpose()
    df.columns = [name, 'Datetime']
    #df.drop('flag', axis = 1, inplace = True)
    #for i, rows in df.iterrows():
        #print(i)
        #print(df['Datetime'][i].ToString())
    #    df['Datetime'][i] = df['Datetime'][i].ToString()
        
    #df['Datetime'] = pd.to_datetime(df['Datetime'])
    #df.set_index(['Datetime'])
    
    fname = sys.path[0] + '\data\K-EO601\\' + year + '_' + name + '.csv'
    df.to_csv(fname, header = False, index = False)
    del df
    toc = time.perf_counter()
    print('Wrote ' + name + f" in {toc - tic:0.1f} seconds")
    #df_list.append(df)
    
#full_dataset = pd.concat(df_list, axis = 1)
#full_dataset = full_dataset.loc[:,~full_dataset.columns.duplicated()]
#full_dataset = reduce(lambda  left, right: pd.merge(left, right,on = ['Datetime'], how = 'outer'), df_list)
#full_dataset.set_index('Datetime')

end = time.perf_counter()
print('Done.  ' + f"Total Duration: {(end - start)/60:0.1f} minutes")

Wrote xeo6401az_eo2 in 9.6 seconds
Wrote feo602_eo2 in 29.8 seconds
Wrote peo6414d_eo2 in 31.3 seconds
Wrote seo603_eo2 in 29.1 seconds
Wrote seo602_eo2 in 27.2 seconds
Wrote teo6422_eo2 in 29.8 seconds
Wrote teo6423_eo2 in 27.8 seconds
Wrote jeo601_eo2 in 28.1 seconds
Wrote neo6012x_eo2 in 28.7 seconds
Wrote neo6012y_eo2 in 26.3 seconds
Wrote neo6011x_eo2 in 27.5 seconds
Wrote neo6011y_eo2 in 27.4 seconds
Done.  Total Duration: 5.4 minutes


In [20]:
# Build the tag index
tag_index = pd.DataFrame(pi_tags, columns = ['pitag'])
desc_list = [];
eng_units = [];

for pitag in tag_index['pitag']:
    pt = PIPoint.FindPIPoint(piServer, pitag)
    pt.LoadAttributes()
    desc_list.append(pt.GetAttribute('descriptor'))
    eng_units.append(pt.GetAttribute('engunits'))

tag_index['descriptor'] = desc_list
tag_index['eng units'] = eng_units

# Show me the dataframe
tag_index

,pitag,descriptor,eng units
0,XEO6401AZ_EO2,XV-EO6401A VALVE POSTION (I-5b),0 = N2
1,FEO602_EO2,REC LOOP VENT,MLB/HR
2,PEO6414D_EO2,K-EO601 DRY GAS SEAL PRIMARY TO SECONDARY DIFF...,PSID
3,SEO603_EO2,GK-EO601 LOW SPEED CASING ACCELEROMETER,G PK
4,SEO602_EO2,GK-EO601 HIGH SPEED CASING ACCELEROMETER,G PK
5,TEO6422_EO2,DK-EO601 AIR CIRCULATION INLET TEMP,DEGF
6,TEO6423_EO2,DK-EO601 AIR CIRCULATION OUTLET TEMP,DEGF
7,JEO601_EO2,DK-EO601A MOTOR POWER,KW
8,NEO6012X_EO2,DK-EO601A MOTOR NON DRIVE END JOURNAL BEARING ...,MILS P-P
9,NEO6012Y_EO2,DK-EO601A MOTOR NON DRIVE END JOURNAL BEARING ...,MILS P-P


In [21]:
# Write it to CSV
fname = sys.path[0] + '\data\K-EO601\\' + 'tag_index' + '.csv'
tag_index.to_csv(fname)

In [9]:
#i'm not getting good results with wildcard yet
pt = PIPoint.FindPIPoint(piServer, 'PEO671A_EO2')
# Commands I ran through to get the attribute I'm after
#print(dir(pt))
#print(dir(pt.FindAttributeNames))
pt.FindAttributeNames.__init__()
print(dir(pt.GetAttributes))
pt.FindAttributeNames.__call__('')
# These two are key
pt.GetAttributes.__init__()
pt.GetAttribute('descriptor')

['__call__', '__class__', '__delattr__', '__delete__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__set__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


'K-EO601 DIFF PRESSURE (I-1)'